<a href="https://colab.research.google.com/github/GustavinhoDantas/LeituraPDFpy/blob/main/LeitorPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LEITOR DE PDF**
A ideia é pegar o PDF e ler de maneira automatica, onde o valor lidro ira retornar como arquivo excel ou SQL

In [ ]:

!pip install pdfplumber pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 86.2 MB/s eta 0:00:00


In [ ]:
import pdfplumber
import re
import pandas as pd

# Lista para armazenar os dicionários dos dados extraídos
dados = []

with pdfplumber.open('/content/Leo_Teste_PDF.pdf') as pdf:
    texto_completo = ''
    for pagina in pdf.pages:
        texto_completo += pagina.extract_text() + '\n'

    # Quebra o texto em blocos usando a linha cheia de pontos
    blocos = re.split(r'\n\.{10,}\n', texto_completo)

    for bloco in blocos:
     if 'Placa.......:' in bloco:
        # Tratamento especial para a Placa
        match = re.search(r'Placa\.*:\s*(.{7})', bloco)
        placa = match.group(1).strip() if match else ''

        # Tratamento especial para a Marca
        match = re.search(r'Marca\.*:\s*(.*?)(?=\bAno\b)', bloco)
        marca = match.group(1).strip() if match else ''

        # Tratamento especial para a Proprietário
        match = re.search(r'Prop. Atual\.*:\s*(.*)', bloco)
        proprietario = match.group(1).strip()[:30] if match else ''

        # Extrair telefones, só os 11 primeiros números
        match = re.search(r'Fone Res\.*:\s*(.{13})', bloco)
        telefone_res = ''.join(filter(str.isdigit, match.group(1)))[:11] if match else ''

        match = re.search(r'Fone Com\.*:\s*(.{13})', bloco)
        telefone_com = ''.join(filter(str.isdigit, match.group(1)))[:11] if match else ''

        match = re.search(r'Fone Cel\.*:\s*(.{13})', bloco)
        telefone_cel = ''.join(filter(str.isdigit, match.group(1)))[:11] if match else ''

        # Extrair origem sendo ele a linha toda da leitura
        #####VERIFICAR SE PODE PARAR DE LER AO IDENTIFICAR ESPAÇO (VAZIO)
        match = re.search(r'Origem\.*:\s*(.*)', bloco)
        origem = match.group(1).strip()[:30] if match else ''

        # Extrair exercicios, sendo ele sempre quatro digitos após :
        match = re.search(r'Exer:\s*(.{4})', bloco)
        exercicio = match.group(1).strip() if match else ''

        dados.append({
            'PLACA': placa,
            'MARCA': marca,
            'ORIGEM': origem,
            'PROPRIETÁRIO ATUAL': proprietario,
            'TELEFONE RESPONSÁVEL': telefone_res,
            'TELEFONE CELULAR': telefone_cel,
            'TELEFONE COMERCIAL': telefone_com,
            'EXERCÍCIO': exercicio,
        })

# Transforma em DataFrame e salva
df = pd.DataFrame(dados)
df.to_excel('dados_extraidosaqui.xlsx', index=False)
print('Dados extraídos e salvos no Excel!')

Dados extraídos e salvos no Excel!
